In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
df = yf.download("MSFT", start='2020-01-01', end='2020-12-31')
df.head()

/tmp/ipython-input-2054940203.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("MSFT", start='2020-01-01', end='2020-12-31')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
Date,,,,,
2020-01-02,152.791092,152.895731,150.612717,151.040780,22622100
2020-01-03,150.888580,152.153756,150.355878,150.603214,21116200
2020-01-06,151.278641,151.345236,148.881465,149.423690,20813700
2020-01-07,149.899307,151.887434,149.651985,151.554502,21634100
2020-01-08,152.286987,152.962387,150.251294,151.183524,27746500


In [ ]:
df.columns

MultiIndex([( 'Close', 'MSFT'),
            (  'High', 'MSFT'),
            (   'Low', 'MSFT'),
            (  'Open', 'MSFT'),
            ('Volume', 'MSFT')],
           names=['Price', 'Ticker'])

In [ ]:
df = yf.download(tickers=['AAPL', 'MSFT'], start='2020-01-01', end='2020-12-31', progress=)
df.head()

[*********************100%***********************]  2 of 2 completed


Price           Close                   High                    Low  \
Ticker           AAPL        MSFT       AAPL        MSFT       AAPL   
Date                                                                  
2020-01-02  72.538506  152.791122  72.598884  152.895761  71.292296   
2020-01-03  71.833305  150.888565  72.594071  152.153740  71.608700   
2020-01-06  72.405663  151.278595  72.444306  151.345190  70.702997   
2020-01-07  72.065147  149.899277  72.671341  151.887403  71.845369   
2020-01-08  73.224388  152.286957  73.526280  152.962357  71.768064   

Price                        Open                 Volume            
Ticker            MSFT       AAPL        MSFT       AAPL      MSFT  
Date                                                                
2020-01-02  150.612747  71.545882  151.040810  135480400  22622100  
2020-01-03  150.355863  71.765682  150.603199  146322800  21116200  
2020-01-06  148.881420  70.954173  149.423644  118387200  20813700  
2020-01-07  149.651955  72.415337  151.554471  108872000  21634100  
2020-01-08  150.251264  71.768064  151.183494  132079200  27746500

In [ ]:
import pandas as pd
import yfinance as yf

In [ ]:
def _normalize_multi_ticker_frame(df: pd.DataFrame) -> dict:
    """
    Accepts yfinance batch download output in either MultiIndex layout:
      1) ('Price','Ticker')  e.g., ('Close','AAPL')
      2) ('Ticker','Price')  e.g., ('AAPL','Close')
    or a single-index DataFrame.

    Returns dict[ticker] -> DataFrame with columns:
      ['Open','High','Low','Close','Adj Close','Volume'] (Adj Close = Close if missing)
    Index is tz-naive DatetimeIndex.
    """
    out = {}
    if df is None or df.empty:
        return out

    def _ensure_cols(one: pd.DataFrame) -> pd.DataFrame:
        one = one.copy()
        # make sure required columns exist
        for c in ["Open","High","Low","Close","Volume"]:
            if c not in one.columns:
                one[c] = np.nan
        if "Adj Close" not in one.columns or one["Adj Close"].isna().all():
            one["Adj Close"] = one["Close"]
        # numeric & tz-naive index
        for c in ["Open","High","Low","Close","Adj Close","Volume"]:
            one[c] = pd.to_numeric(one[c], errors="coerce")
        if getattr(one.index, "tz", None) is not None:
            one = one.tz_convert(None)
        one.index = pd.DatetimeIndex(one.index).tz_localize(None)
        return one[["Open","High","Low","Close","Adj Close","Volume"]]

    # MultiIndex case
    if isinstance(df.columns, pd.MultiIndex):
        lvl0 = df.columns.get_level_values(0)
        lvl1 = df.columns.get_level_values(1)

        # Detect which level is ticker vs price field
        price_fields = {"Open","High","Low","Close","Adj Close","Volume"}

        # If level 0 looks like price fields -> layout is ('Price','Ticker')
        if set(lvl0) & price_fields:
            # Convert to ('Ticker','Price') so we can slice by ticker cleanly
            df2 = df.swaplevel(0, 1, axis=1).sort_index(axis=1)
        else:
            # Already ('Ticker','Price') or something equivalent
            df2 = df.sort_index(axis=1)

        tickers = df2.columns.get_level_values(0).unique()
        for t in tickers:
            sub = df2[t]
            out[t] = _ensure_cols(sub)

    else:
        # Single-index (single ticker)
        one = _ensure_cols(df.copy())
        out["__SINGLE__"] = one

    return out

DJIA_TICKERS = [
    'AAPL','MSFT','JPM','V','UNH','HD','PG','JNJ','CVX','MRK',
    'ABBV','BAC','PEP','COST','XOM','KO','CSCO','WMT','MCD','TMO',
    'ACN','ABT','DIS','DHR','VZ','ADBE','NFLX','NKE','CRM','INTC'
]

INDUSTRY_MAP = {
    'Technology': ['AAPL', 'MSFT', 'CSCO', 'INTC', 'CRM', 'ADBE'],
    'Healthcare': ['JNJ', 'UNH', 'MRK', 'ABBV', 'TMO', 'ABT', 'DHR'],
    'Financial': ['JPM', 'V', 'BAC'],
    'Consumer Discretionary': ['HD', 'MCD', 'DIS', 'NKE', 'NFLX'],
    'Consumer Staples': ['PG', 'PEP', 'KO', 'WMT', 'COST'],
    'Energy': ['CVX', 'XOM'],
    'Telecom': ['VZ'],
    'Industrials': ['ACN'],
}
# data = yf.download(tickers=DJIA_TICKERS, start='2020-01-01', end='2020-12-31', progress=False)
# norm = _normalize_multi_ticker_frame(data)

In [ ]:
def download_market_data(start, end):
  dji = yf.download("^DJI", start=start, end=end, progress=False)
  vix = yf.download("^VIX", start=start, end=end, progress=False)
  tnx = yf.download("^TNX", start=start, end=end, progress=False)
  return dji, vix, tnx

def norm_market_data(dji, vix, tnx):
  dji_norm = _normalize_multi_ticker_frame(dji)
  vix_norm = _normalize_multi_ticker_frame(vix)
  tnx_norm = _normalize_multi_ticker_frame(tnx)
  return dji_norm, vix_norm, tnx_norm

In [ ]:
data = yf.download("^DJI", start='2020-01-01', end='2020-12-31', progress=False,ignore_tz=True)
data.head()

Price,Close,High,Low,Open,Volume
Ticker,^DJI,^DJI,^DJI,^DJI,^DJI
Date,,,,,
2020-01-02,28868.800781,28872.800781,28627.769531,28638.970703,251820000
2020-01-03,28634.880859,28716.310547,28500.359375,28553.330078,239590000
2020-01-06,28703.380859,28708.019531,28418.630859,28465.500000,252760000
2020-01-07,28583.679688,28685.500000,28565.279297,28639.179688,258900000
2020-01-08,28745.089844,28866.179688,28522.509766,28556.140625,291750000


In [ ]:
def norm_to_stock_dict(norm: dict, tickers: list, market_dates: pd.DatetimeIndex) -> dict:
    """
    Turn norm -> aligned_stock dict[ticker] -> DataFrame
    - norm: output of _normalize_multi_ticker_frame
    - tickers: list of desired tickers (final order)
    - market_dates: DatetimeIndex to reindex to (tz-naive)
    """
    aligned = {}
    for t in tickers:
        if t in norm:
            df = norm[t].copy()
        else:
            # create empty frame with expected index and columns if ticker missing
            cols = ["Open","High","Low","Close","Adj Close","Volume"]
            df = pd.DataFrame(index=market_dates, columns=cols, dtype=float)
            print(f"  INFO: {t} missing in norm; creating empty placeholder")
        # Ensure tz-naive index
        df.index = pd.DatetimeIndex(df.index).tz_localize(None) if getattr(df.index, "tz", None) is None else df.tz_convert(None).tz_localize(None)
        # Reindex to market calendar, then forward/back-fill to avoid NaNs for numeric arrays
        df = df.reindex(market_dates)
        df = df.ffill().bfill()
        # Ensure numeric columns & exact ordering
        for c in ["Open","High","Low","Close","Adj Close","Volume"]:
            if c not in df.columns:
                df[c] = np.nan
            df[c] = pd.to_numeric(df[c], errors="coerce").astype(float)
        aligned[t] = df[["Open","High","Low","Close","Adj Close","Volume"]]
    return aligned

def calculate_sma5(series):
    return series.rolling(5, min_periods=1).mean()

In [ ]:
data = yf.download(tickers=DJIA_TICKERS, start='2020-01-01', end='2020-12-31', progress=False)
norm = _normalize_multi_ticker_frame(data)

# 2) get market series (so we know market_dates)
dji, vix, tnx = download_market_data('2020-01-01', '2020-12-31')

# 3) decide market_dates exactly like your align_dates does (reuse that logic or compute simple intersection)
market_dates = pd.DatetimeIndex(dji.index).tz_localize(None)
market_dates = market_dates.intersection(pd.DatetimeIndex(vix.index).tz_localize(None)).intersection(pd.DatetimeIndex(tnx.index).tz_localize(None))
if len(market_dates) < 500:
    # fallback policy (same as your align_dates)
    market_dates = pd.DatetimeIndex(dji.index).tz_localize(None).intersection(pd.DatetimeIndex(vix.index).tz_localize(None))
if len(market_dates) < 500:
    market_dates = pd.DatetimeIndex(dji.index).tz_localize(None)
market_dates = pd.DatetimeIndex(sorted(market_dates))

# 4) align/normalize tickers to market calendar
aligned_stock = norm_to_stock_dict(norm, DJIA_TICKERS, market_dates)
# 6) final tickers order
final_tickers = sorted(aligned_stock.keys())

# 7) create arrays (reuse your existing functions)
# X = create_stocks_data(aligned_stock, final_tickers)
# R = create_ror_data(aligned_stock, final_tickers)
# M = create_market_data(dji.reindex(market_dates).ffill().bfill(),
#                        vix.reindex(market_dates).ffill().bfill(),
#                        tnx.reindex(market_dates).ffill().bfill())
# G = create_industry_classification(final_tickers)

# # 8) validate & save
# validate_data(X, R, M, G)
# save_data(X, R, M, G, "./data/DJIA/")

/tmp/ipython-input-3777685433.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=DJIA_TICKERS, start='2020-01-01', end='2020-12-31', progress=False)
/tmp/ipython-input-2480530143.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dji = yf.download("^DJI", start=start, end=end, progress=False)
/tmp/ipython-input-2480530143.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  vix = yf.download("^VIX", start=start, end=end, progress=False)
/tmp/ipython-input-2480530143.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  tnx = yf.download("^TNX", start=start, end=end, progress=False)


In [ ]:
import os
import numpy as np
import pandas as pd

def create_stocks_data(aligned_stock_data: dict, tickers: list) -> np.ndarray:
    """
    aligned_stock_data: dict[ticker] -> DataFrame indexed by market_dates with columns
                        ['Open','High','Low','Close','Adj Close','Volume']
    tickers: list of tickers in the final order
    Returns X: np.array shape (n_tickers, T, 6) with columns:
      0: Open, 1: High, 2: Low, 3: Adj Close, 4: Volume, 5: SMA_5
    """
    print("Building stocks_data.npy ...")
    n = len(tickers)
    if n == 0:
        raise ValueError("Empty tickers list passed to create_stocks_data")
    T = len(next(iter(aligned_stock_data.values())))
    X = np.zeros((n, T, 6), dtype=np.float64)

    for i, t in enumerate(tickers):
        df = aligned_stock_data[t].copy()
        # technical indicator
        price = df["Adj Close"].astype(float)
        df["SMA_5"] = price.rolling(5, min_periods=1).mean()
        # Fill any remaining NaNs defensively (shouldn't exist if alignment used ffill/bfill)
        df = df[["Open","High","Low","Adj Close","Volume","SMA_5"]].astype(float).ffill().bfill().fillna(0.0)
        X[i, :, 0] = df["Open"].values
        X[i, :, 1] = df["High"].values
        X[i, :, 2] = df["Low"].values
        X[i, :, 3] = df["Adj Close"].values
        X[i, :, 4] = df["Volume"].values
        X[i, :, 5] = df["SMA_5"].values

    print("  shape:", X.shape)
    return X


def create_ror_data(aligned_stock_data: dict, tickers: list) -> np.ndarray:
    """
    Create returns-on-risk (simple returns) matrix R of shape (n_tickers, T).
    ror[0] = 0 (no prior), ror[t] = (p[t] - p[t-1]) / p[t-1], with denom guarded.
    """
    print("Building ror.npy ...")
    n = len(tickers)
    if n == 0:
        raise ValueError("Empty tickers list passed to create_ror_data")
    T = len(next(iter(aligned_stock_data.values())))
    R = np.zeros((n, T), dtype=np.float64)

    for i, t in enumerate(tickers):
        p = aligned_stock_data[t]["Adj Close"].astype(float).values
        # guard: if all zeros, keep zeros
        if len(p) < 2:
            R[i, :] = 0.0
            continue
        denom = np.where(p[:-1] == 0.0, 1.0, p[:-1])
        ror = np.zeros(T, dtype=np.float64)
        ror[1:] = (p[1:] - p[:-1]) / denom
        R[i, :] = ror

    print("  shape:", R.shape)
    return R


def create_market_data(dji: pd.DataFrame, vix: pd.DataFrame, tnx: pd.DataFrame) -> np.ndarray:
    """
    dji, vix, tnx: DataFrames aligned to the same index (market_dates) with at least 'Close' column.
    Produces M: np.array shape (T, 4) columns:
      0: DJI Close, 1: DJI Volume (or 0), 2: VIX Close, 3: TNX Close
    """
    print("Building market_data.npy ...")
    # Ensure indices align and determine T
    if not (len(dji) == len(vix) == len(tnx)):
        # try reindexing to the intersection/union; prefer dji index as canonical
        idx = pd.DatetimeIndex(dji.index)
        vix = vix.reindex(idx).ffill().bfill()
        tnx = tnx.reindex(idx).ffill().bfill()

    T = len(dji)
    M = np.zeros((T, 4), dtype=np.float64)

    # Column safe extraction (use 0 for missing numeric values)
    M[:, 0] = pd.to_numeric(dji.get("Close", pd.Series(np.nan, index=dji.index)), errors="coerce").fillna(0.0).values
    if "Volume" in dji.columns:
        M[:, 1] = pd.to_numeric(dji["Volume"], errors="coerce").fillna(0.0).values
    else:
        M[:, 1] = 0.0
    M[:, 2] = pd.to_numeric(vix.get("Close", pd.Series(np.nan, index=dji.index)), errors="coerce").fillna(0.0).values
    M[:, 3] = pd.to_numeric(tnx.get("Close", pd.Series(np.nan, index=dji.index)), errors="coerce").fillna(0.0).values

    # final defensive ffill/bfill in case of any NaNs
    M = pd.DataFrame(M).ffill().bfill().values
    print("  shape:", M.shape)
    return M


def create_industry_classification(tickers: list, industry_map: dict) -> np.ndarray:
    """
    Build an industry adjacency/affinity matrix G of shape (n, n).
    Diagonal = 1.0; off-diagonal = 1.0 if same industry, else 0.0.
    industry_map: dict[industry_name] -> list[tickers]
    """
    print("Building industry_classification.npy ...")
    n = len(tickers)
    G = np.eye(n, dtype=np.float64)
    # map ticker -> industry
    t2industry = {}
    for industry, lst in industry_map.items():
        for tk in lst:
            t2industry[tk] = industry
    for i, ti in enumerate(tickers):
        for j, tj in enumerate(tickers):
            if i == j:
                continue
            if t2industry.get(ti, None) is not None and t2industry.get(ti) == t2industry.get(tj):
                G[i, j] = 1.0
    print(f"  shape: {G.shape}, connections(off-diag): {int(G.sum() - n)}")
    return G


def validate_data(stocks_data: np.ndarray, ror_data: np.ndarray, market_data: np.ndarray, industry_matrix: np.ndarray):
    """
    Basic validation checks (shapes, NaN/Inf).
    Raises an AssertionError or ValueError on failure.
    """
    print("\nValidating ...")
    assert stocks_data.shape[:-1] == ror_data.shape, "stocks_data vs ror shape mismatch"
    assert stocks_data.shape[1] == market_data.shape[0], "time axis mismatch"
    assert industry_matrix.shape[0] == industry_matrix.shape[1], "industry matrix not square"
    assert industry_matrix.shape[0] == stocks_data.shape[0], "industry size vs tickers mismatch"

    for name, arr in [('stocks_data', stocks_data), ('ror_data', ror_data),
                      ('market_data', market_data), ('industry_matrix', industry_matrix)]:
        if np.isnan(arr).any():
            raise ValueError(f"{name} has NaN")
        if np.isinf(arr).any():
            raise ValueError(f"{name} has Inf")
    print("  ✓ OK")
    print("Stats:")
    print(f"  stocks_data min/max/mean: {stocks_data.min():.4f}/{stocks_data.max():.4f}/{stocks_data.mean():.4f}")
    print(f"  ror_data    min/max/mean: {ror_data.min():.6f}/{ror_data.max():.6f}/{ror_data.mean():.6f}")
    print(f"  market_data min/max/mean: {market_data.min():.4f}/{market_data.max():.4f}/{market_data.mean():.4f}")


def save_data(stocks_data: np.ndarray, ror_data: np.ndarray, market_data: np.ndarray, industry_matrix: np.ndarray, output_dir: str):
    """
    Save numpy arrays to output_dir (creates directory if needed).
    """
    print(f"\nSaving to {output_dir} ...")
    os.makedirs(output_dir, exist_ok=True)
    np.save(os.path.join(output_dir, "stocks_data.npy"), stocks_data)
    np.save(os.path.join(output_dir, "ror.npy"), ror_data)
    np.save(os.path.join(output_dir, "market_data.npy"), market_data)
    np.save(os.path.join(output_dir, "industry_classification.npy"), industry_matrix)
    print("  ✓ saved")
    print("\nGenerated files:")
    print(f"  - {os.path.join(output_dir, 'stocks_data.npy')}")
    print(f"  - {os.path.join(output_dir, 'ror.npy')}")
    print(f"  - {os.path.join(output_dir, 'market_data.npy')}")
    print(f"  - {os.path.join(output_dir, 'industry_classification.npy')}")

In [ ]:
type(dji)

pandas.core.frame.DataFrame

In [ ]:
dji_norm, vix_norm, tnx_norm = norm_market_data(dji, vix, tnx)
# type(dji_norm)

def collapse_market_multiindex(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert a yfinance market DataFrame with possible MultiIndex columns into a
    single-level df with columns ['Open','High','Low','Close','Adj Close','Volume'].
    If Adj Close is missing it is copied from Close.
    Ensures tz-naive DatetimeIndex.
    """
    if df is None or df.empty:
        return pd.DataFrame(columns=["Open","High","Low","Close","Adj Close","Volume"])

    out = df.copy()

    # If MultiIndex, try to pick the correct layout
    if isinstance(out.columns, pd.MultiIndex):
        lvl0 = out.columns.get_level_values(0)
        lvl1 = out.columns.get_level_values(1)

        # detect whether level names/values look like price fields or tickers
        price_fields = set(["Open","High","Low","Close","Adj Close","Volume"])
        # layout ('Price','Ticker') -> lvl0 contains price names
        if set(lvl0) & price_fields:
            # swap to (Ticker, Price) so out[ticker][price] works
            out = out.swaplevel(0,1,axis=1).sort_index(axis=1)

        # Now we expect columns like (Ticker, Price). Pick the first ticker column.
        # If there are multiple tickers, prefer the one whose name matches a known index symbol.
        # But for market indices like '^DJI' there's usually only one ticker column.
        tickers = out.columns.get_level_values(0).unique()
        # pick first ticker available
        primary = tickers[0]
        out = out[primary]  # now a DataFrame with price-field columns

    # After collapsing, ensure columns exist and are Series (not DataFrame)
    for c in ["Open","High","Low","Close","Volume"]:
        if c not in out.columns:
            # If out[c] is a DataFrame (multi), try to pick first subcol
            if c in out.columns and isinstance(out[c], pd.DataFrame):
                # take first numeric subcolumn
                for sub in out[c].columns:
                    if pd.api.types.is_numeric_dtype(out[c][sub]):
                        out[c] = out[c][sub]
                        break
                else:
                    out[c] = np.nan
            else:
                out[c] = np.nan

    if "Adj Close" not in out.columns or out["Adj Close"].isna().all():
        out["Adj Close"] = out["Close"]

    # Ensure numeric dtype
    for c in ["Open","High","Low","Close","Adj Close","Volume"]:
        out[c] = pd.to_numeric(out[c], errors="coerce")

    # Make tz-naive index
    if getattr(out.index, "tz", None) is not None:
        out = out.tz_convert(None)
    out.index = pd.DatetimeIndex(out.index).tz_localize(None)

    # Return exact column order
    return out[["Open","High","Low","Close","Adj Close","Volume"]]

In [ ]:
# 7) create arrays (reuse your existing functions)
X = create_stocks_data(aligned_stock, final_tickers)
R = create_ror_data(aligned_stock, final_tickers)
# Usage: call collapse_market_multiindex BEFORE create_market_data
dji_clean = collapse_market_multiindex(dji).reindex(market_dates).ffill().bfill()
vix_clean = collapse_market_multiindex(vix).reindex(market_dates).ffill().bfill()
tnx_clean = collapse_market_multiindex(tnx).reindex(market_dates).ffill().bfill()

M = create_market_data(dji_clean, vix_clean, tnx_clean)
G = create_industry_classification(final_tickers, INDUSTRY_MAP)

# 8) validate & save
validate_data(X, R, M, G)
save_data(X, R, M, G, "./data/DJIA/")

Building stocks_data.npy ...
  shape: (30, 252, 6)
Building ror.npy ...
  shape: (30, 252)
Building market_data.npy ...
  shape: (252, 4)
Building industry_classification.npy ...
  shape: (30, 30), connections(off-diag): 120

Validating ...
  ✓ OK
Stats:
  stocks_data min/max/mean: 15.6723/426510000.0000/3114712.3115
  ror_data    min/max/mean: -0.221248/0.260449/0.000910
  market_data min/max/mean: 0.4990/915990000.0000/105069196.7603

Saving to ./data/DJIA/ ...
  ✓ saved

Generated files:
  - ./data/DJIA/stocks_data.npy
  - ./data/DJIA/ror.npy
  - ./data/DJIA/market_data.npy
  - ./data/DJIA/industry_classification.npy


In [ ]:
!zip -r DJIA_data.zip ./data/DJIA/

  adding: data/DJIA/ (stored 0%)
  adding: data/DJIA/ror.npy (deflated 5%)
  adding: data/DJIA/industry_classification.npy (deflated 97%)
  adding: data/DJIA/stocks_data.npy (deflated 28%)
  adding: data/DJIA/market_data.npy (deflated 50%)


In [ ]:
from google.colab import files
files.download('DJIA_data.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
dji.columns

MultiIndex([( 'Close', '^DJI'),
            (  'High', '^DJI'),
            (   'Low', '^DJI'),
            (  'Open', '^DJI'),
            ('Volume', '^DJI')],
           names=['Price', 'Ticker'])

In [ ]:
df['Close', 'MSFT']

,Close
,MSFT
Date,
2020-01-02,152.791122
2020-01-03,150.888565
2020-01-06,151.278595
2020-01-07,149.899277
2020-01-08,152.286957
...,...
2020-12-23,212.474396
2020-12-24,214.137527


In [ ]:
# ====== DeepTrader Data Prep — simplest yfinance (Colab cell) ======
# - One ticker at a time (serial), minimal flags
# - Skips tickers that fail after a few retries
# - Aligns by common trading days across succeeded tickers
# - Market series (^DJI,^VIX,^TNX) optional: if failing, fills NaNs
# - Saves to /content/data/DJIA/
import time
import os
import numpy as np
import pandas as pd
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

# ---------- USER CONFIG (edit as needed) ----------
MARKET = "DJIA"                  # only DJIA supported in this simple cell
START  = "2025-06-01"
END    = "2025-10-01"
OUTPUT_DIR = "/data/DJIA"  # change if desired
MIN_STOCKS = 10                   # minimum tickers that must succeed (else stop)
FORCE = False                     # if True, proceed even with few dates/tickers
AUTO_ADJUST_STOCKS = True         # yfinance adjusts OHLC; Adj Close may be missing
# -----------------------------------------------

# If you want to save into Google Drive, uncomment:
# from google.colab import drive
# drive.mount('/content/drive')
# OUTPUT_DIR = "/content/drive/MyDrive/DeepTraderData/DJIA"

# ---------- DJIA-ish list (your original set) ----------
DJIA_TICKERS = [
    'AAPL','MSFT','JPM','V','UNH','HD','PG','JNJ','CVX','MRK',
    'ABBV','BAC','PEP','COST','XOM','KO','CSCO','WMT','MCD','TMO',
    'ACN','ABT','DIS','DHR','VZ','ADBE','NFLX','NKE','CRM','INTC'
]

# Simplified sector map
INDUSTRY_MAP = {
    'Technology': ['AAPL','MSFT','CSCO','INTC','CRM','ADBE'],
    'Healthcare': ['JNJ','UNH','MRK','ABBV','TMO','ABT','DHR'],
    'Financial': ['JPM','V','BAC'],
    'Consumer Discretionary': ['HD','MCD','DIS','NKE','NFLX'],
    'Consumer Staples': ['PG','PEP','KO','WMT','COST'],
    'Energy': ['CVX','XOM'],
    'Telecom': ['VZ'],
    'Industrials': ['ACN'],
}

print("="*60)
print("DeepTrader Data Prep — simplest (Colab)")
print("="*60)
print(f"Market : {MARKET}")
print(f"Dates  : {START} → {END}")
print(f"Output : {OUTPUT_DIR}")
print("="*60)

# -------------------- Helpers --------------------
def simple_download(ticker, start, end, auto_adjust=True, max_retries=3):
    """
    Minimal serial download with small retry. No special args.
    Returns a DataFrame with: Open, High, Low, Close, Adj Close, Volume
    or empty DataFrame if fetch fails.
    """
    for attempt in range(1, max_retries+1):
        try:
            df = yf.download(
                tickers=ticker,
                start=start, end=end,
                auto_adjust=auto_adjust,   # if True, OHLC adjusted; Adj Close may be missing
                progress=False,
                threads=False
            )
            if df is None or df.empty:
                time.sleep(0.8 * attempt)
                continue

            # Ensure columns exist; if "Adj Close" missing, use Close
            if "Adj Close" not in df.columns:
                df["Adj Close"] = df["Close"]

            keep = ["Open","High","Low","Close","Adj Close","Volume"]
            for c in keep:
                if c not in df.columns:
                    df[c] = np.nan
                df[c] = pd.to_numeric(df[c], errors="coerce")

            df.index = pd.to_datetime(df.index)
            df.index = pd.DatetimeIndex(df.index).tz_localize(None)
            return df[keep]
        except Exception:
            time.sleep(0.8 * attempt)
    return pd.DataFrame(columns=["Open","High","Low","Close","Adj Close","Volume"])

def download_stock_data(tickers, start, end, auto_adjust=True):
    """
    Serially download each ticker. Skip if empty after retries.
    """
    data = {}
    print(f"Downloading {len(tickers)} tickers (simple serial)...")
    for t in tickers:
        df = simple_download(t, start, end, auto_adjust=auto_adjust)
        if df.empty:
            print(f"  SKIP: {t} (no data)")
        else:
            print(f"  OK  : {t} -> {len(df)} rows")
            data[t] = df
    return data

def try_download_index(symbol, start, end):
    """
    Try to download an index (^DJI, ^VIX, ^TNX) simply.
    Returns DataFrame or empty DataFrame if fails.
    """
    return simple_download(symbol, start, end, auto_adjust=False)

def align_to_common_days(stock_data_dict):
    """
    Align dict[ticker]->DataFrame to the intersection of dates.
    """
    if not stock_data_dict:
        return {}, pd.DatetimeIndex([])
    it = iter(stock_data_dict.values())
    common = next(it).index
    for df in it:
        common = common.intersection(df.index)
    aligned = {t: df.loc[common] for t, df in stock_data_dict.items()}
    return aligned, common

def calculate_sma5(series):
    return series.rolling(5, min_periods=1).mean()

# -------------------- Builders --------------------
def create_stocks_data(aligned_stock_data, tickers, dates_index):
    """
    stocks_data: [num_stocks, num_days, 6]
    Order: Open, High, Low, AdjClose, Volume, SMA_5
    """
    num_stocks = len(tickers)
    num_days = len(dates_index)
    X = np.zeros((num_stocks, num_days, 6), dtype=np.float64)
    for i, t in enumerate(tickers):
        df = aligned_stock_data[t].copy()
        df["SMA_5"] = calculate_sma5(df["Adj Close"])
        X[i,:,0] = df["Open"].values
        X[i,:,1] = df["High"].values
        X[i,:,2] = df["Low"].values
        X[i,:,3] = df["Adj Close"].values
        X[i,:,4] = df["Volume"].values
        X[i,:,5] = df["SMA_5"].values
    return X

def create_ror_data(aligned_stock_data, tickers, dates_index):
    """
    ror.npy: [num_stocks, num_days] — returns from Adj Close
    """
    num_stocks = len(tickers)
    num_days = len(dates_index)
    R = np.zeros((num_stocks, num_days), dtype=np.float64)
    for i, t in enumerate(tickers):
        p = aligned_stock_data[t]["Adj Close"].astype(float).values
        r = np.zeros(num_days, dtype=np.float64)
        if num_days > 1:
            denom = np.where(p[:-1] == 0.0, 1.0, p[:-1])
            r[1:] = (p[1:] - p[:-1]) / denom
        R[i,:] = r
    return R

def create_market_data(dates_index, dji_df, vix_df, tnx_df):
    """
    market_data.npy: [num_days, 4] -> (DJI Close, DJI Volume, VIX Close, TNX Close)
    Any missing series → NaNs.
    """
    num_days = len(dates_index)
    M = np.full((num_days, 4), np.nan, dtype=np.float64)

    def safe_get(df, col):
        if df is None or df.empty or col not in df.columns:
            return np.full(num_days, np.nan, dtype=np.float64)
        s = df[col].reindex(dates_index)
        return s.values.astype(np.float64)

    M[:,0] = safe_get(dji_df, "Close")
    M[:,1] = safe_get(dji_df, "Volume")
    M[:,2] = safe_get(vix_df, "Close")
    M[:,3] = safe_get(tnx_df, "Close")
    return M

def create_industry_classification(tickers):
    """
    [num_stocks, num_stocks], 1 if same mapped sector (diagonal 1s).
    """
    n = len(tickers)
    G = np.eye(n, dtype=np.float64)
    t2i = {}
    for k, lst in INDUSTRY_MAP.items():
        for t in lst:
            t2i[t] = k
    for i, ti in enumerate(tickers):
        for j, tj in enumerate(tickers):
            if i == j:
                continue
            if t2i.get(ti, 'Unknown') == t2i.get(tj, 'Unknown') != 'Unknown':
                G[i, j] = 1.0
    return G

def validate_and_save(X, R, M, G, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    assert X.shape[:-1] == R.shape, "stocks_data and ror shape mismatch"
    assert X.shape[1] == M.shape[0], "time axis mismatch"
    assert G.shape[0] == G.shape[1] == X.shape[0], "industry matrix size mismatch"

    np.save(f"{out_dir.rstrip('/')}/stocks_data.npy", X)
    np.save(f"{out_dir.rstrip('/')}/ror.npy", R)
    np.save(f"{out_dir.rstrip('/')}/market_data.npy", M)
    np.save(f"{out_dir.rstrip('/')}/industry_classification.npy", G)

    print("\nSaved:")
    print(f"  - {out_dir.rstrip('/')}/stocks_data.npy")
    print(f"  - {out_dir.rstrip('/')}/ror.npy")
    print(f"  - {out_dir.rstrip('/')}/market_data.npy")
    print(f"  - {out_dir.rstrip('/')}/industry_classification.npy")

# -------------------- Run pipeline in this cell --------------------
if MARKET != "DJIA":
    raise SystemExit("This simple Colab cell supports only DJIA for now.")

tickers = DJIA_TICKERS

# 1) Stocks
stock_data = download_stock_data(tickers, START, END, auto_adjust=AUTO_ADJUST_STOCKS)
if not stock_data:
    raise SystemExit("ERROR: all downloads failed.")

good_tickers = sorted(stock_data.keys())
if len(good_tickers) < MIN_STOCKS and not FORCE:
    raise SystemExit(f"ERROR: only {len(good_tickers)} tickers succeeded (<{MIN_STOCKS}). "
                     f"Set FORCE=True to proceed anyway.")

# 2) Align
aligned_stocks, common_dates = align_to_common_days(stock_data)
if len(common_dates) < 200 and not FORCE:
    raise SystemExit(f"ERROR: only {len(common_dates)} common days across succeeded tickers. "
                     f"Set FORCE=True to proceed anyway.")

print(f"Using {len(good_tickers)} tickers and {len(common_dates)} common days.")

# 3) Market series (optional; continue if NaN)
dji = try_download_index("^DJI", START, END)
vix = try_download_index("^VIX", START, END)
tnx = try_download_index("^TNX", START, END)

# 4) Build arrays
X = create_stocks_data(aligned_stocks, good_tickers, common_dates)
R = create_ror_data(aligned_stocks, good_tickers, common_dates)
M = create_market_data(common_dates, dji, vix, tnx)
G = create_industry_classification(good_tickers)

# 5) Validate & save
validate_and_save(X, R, M, G, OUTPUT_DIR)

print("\nDone. Example next step:")
print("  python run.py -c hyper.json")


DeepTrader Data Prep — simplest (Colab)
Market : DJIA
Dates  : 2025-06-01 → 2025-10-01
Output : /data/DJIA
  SKIP: AAPL (no data)
  SKIP: MSFT (no data)
  SKIP: JPM (no data)
  SKIP: V (no data)
  SKIP: UNH (no data)
  SKIP: HD (no data)
  SKIP: PG (no data)
  SKIP: JNJ (no data)
  SKIP: CVX (no data)
  SKIP: MRK (no data)
  SKIP: ABBV (no data)
  SKIP: BAC (no data)
  SKIP: PEP (no data)
  SKIP: COST (no data)
  SKIP: XOM (no data)


KeyboardInterrupt: 